In [1]:
#default_exp retrieval_results

In [16]:
#export
import numpy as np
import pandas as pd

import attr
import pickle

from sklearn import metrics

from github_search.matching_zsl import *

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [47]:
#export


@attr.s
class Retriever:
    
    input_embedder = attr.ib()
    query_embedder = attr.ib()
    zs_learner = attr.ib()
    embeddings_calculated = attr.ib(default=False)
    
    def set_embeddings(self, X_names, X):
        self.X_embeddings = self.input_embedder.transform(X)
        self.X = X
        self.X_names = X_names
        self.embeddings_calculated = True
        
    def retrieve_query_results(self, query, k=25, similarity=metrics.pairwise.cosine_similarity):
        if not self.embeddings_calculated:
            raise Exception("embeddings not calculated")
        input_embeddings = self.X_embeddings
        y_embeddings = self.query_embedder.transform([query])
        predictions = self.zs_learner.predict_raw(input_embeddings)
        input_target_similarities = similarity(predictions, y_embeddings)
        top_idxs = np.argsort(-input_target_similarities[:,0])[:k]
        top_similarities = input_target_similarities[top_idxs, 0]
        top_names = self.X_names.iloc[top_idxs]
        return pd.DataFrame({"matched_record": top_names, "similarity": top_similarities}).reset_index(drop=True)

    def from_retriever_learner(learner):
        return Retriever(learner.input_embedder, learner.y_embedder, learner.zs_learner)

In [48]:
readme_data_test = pickle.load(open("output/readme_data_test.pkl", "rb"))

In [49]:
readme_learner = pickle.load(open("output/readme_learner.pkl", "rb"))

In [50]:
readme_retriever = Retriever.from_retriever_learner(readme_learner)
readme_retriever.set_embeddings(readme_data_test.repos, readme_data_test.X)

In [64]:
distance_learning_results = readme_retriever.retrieve_query_results("similarity learning")

In [65]:
metric_learning_results = readme_retriever.retrieve_query_results("metric learning")

In [66]:
set(distance_learning_results['matched_record']).intersection(set(metric_learning_results['matched_record']))

{'alessiabertugli/FUSION',
 'charliezhaoyinpeng/PDFM',
 'dibyaghosh/gcsl',
 'gdahia/meta_occ',
 'notdibya/gcsl',
 'sfujim/BCQ',
 'valeriechen/ask-your-humans',
 'yuruntian/HyNet'}

In [67]:
word_embedding_results = readme_retriever.retrieve_query_results("word embeddings")

In [68]:
word_embedding_results